In [19]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

sys.path.insert(0, "../../../")


# Evaluation mode

In [20]:
# package for download file from google drive
# ! pip uninstall gdown

In [21]:
# https://drive.google.com/file/d/1yEolUJYvk93S54c1b8zfE22ugVRy06GF/view?usp=drive_link
# "https://drive.google.com/uc?id=0B9P1L--7Wd2vVlpzRkwzYUladms"

In [22]:
folder_name = 'resnet_artefacts'
resnet_artefact_files = {'resnet18_pretrain_simclr':'1yEolUJYvk93S54c1b8zfE22ugVRy06GF',
                         'resnet18_finetune':'1N8CdPJSOBqlb2hzceztXyhrQrxXahg2M'}

In [11]:

# download and extract model files
# os.system('gdown https://drive.google.com/uc?id={}'.format(resnet_artefact_files['resnet18_finetune']))
# os.system('unzip {}'.format(folder_name))
# !ls

import gdown

drive_url = 'https://drive.google.com/file/d/1yEolUJYvk93S54c1b8zfE22ugVRy06GF/view?usp=drive_link'
url = 'https://drive.google.com/uc?id=1yEolUJYvk93S54c1b8zfE22ugVRy06GF'
output = 'resnet_pretrained.tar'
gdown.download(url, output, quiet=False)
# gdown.cached_download(url, output, postprocess=gdown.extractall)
! ls

Downloading...
From (uriginal): https://drive.google.com/uc?id=1yEolUJYvk93S54c1b8zfE22ugVRy06GF
From (redirected): https://drive.google.com/uc?id=1yEolUJYvk93S54c1b8zfE22ugVRy06GF&confirm=t&uuid=5478eba2-79db-432d-aac2-f4d67c5e2e11
To: /home/kgamegah/Documents/academic/UPC/DeepLearning/s3ima/s3ima/arch/SimCLR/prototype/resnet_pretrained.tar
100%|██████████| 138M/138M [00:03<00:00, 42.2MB/s] 


eval.ipynb  resnet_pretrained.tar


In [44]:
from s3ima.arch.ResNet18.model import ResNet18, BasicBlock
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision import datasets
import torch
from collections import Counter
from sklearn.model_selection import train_test_split

In [55]:
mnist_train_dataset = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

mnist_test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    download=False,
    transform=transforms.ToTensor()
)

print(len(mnist_train_dataset))
print(mnist_train_dataset.targets.numpy())
# sample100 = Subset(dataset=mnist_train_dataset,indices=)
Counter(mnist_train_dataset.targets.numpy())

60000
[5 0 4 ... 5 6 8]


Counter({1: 6742,
         7: 6265,
         3: 6131,
         2: 5958,
         9: 5949,
         0: 5923,
         6: 5918,
         8: 5851,
         4: 5842,
         5: 5421})

In [65]:
# sample 100 data with same classes proportion 
train_idx, validation_idx = train_test_split(np.arange(len(mnist_train_dataset)),
                                             train_size=100,
                                             random_state=1,
                                             shuffle=True,
                                             stratify=mnist_train_dataset.targets.numpy())
# Subset dataset for train and val
train_dataset = Subset(mnist_train_dataset, train_idx)


train_loader = DataLoader(train_dataset, 
                          batch_size=10,
                          num_workers=8, 
                          drop_last=False, 
                          shuffle=True)

test_loader = DataLoader(mnist_test_dataset, 
                         batch_size=256,
                         num_workers=8, 
                         drop_last=False, 
                         shuffle=True)

# Counter(train_dataset.targets.numpy())
# train_loader.batch_size

In [66]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cpu


In [67]:
checkpoint = torch.load('resnet_pretrained.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):

    if k.startswith('backbone.'):
        if k.startswith('backbone') and not k.startswith('backbone.fc'):
            # remove prefix
            state_dict[k[len("backbone."):]] = state_dict[k]
    del state_dict[k]

In [68]:
 model = ResNet18(num_layers=18,
                  block=BasicBlock,
                  num_classes=10,
                  grayscale=True)
model.to(device=device)


log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']


In [69]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [70]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [72]:
epochs = 100
for epoch in range(epochs):
    top1_train_accuracy = 0
    for counter, (x_batch, y_batch) in enumerate(train_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        top1 = accuracy(logits, y_batch, topk=(1,))
        top1_train_accuracy += top1[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    top1_train_accuracy /= (counter + 1)
    top1_accuracy = 0
    top5_accuracy = 0
    for counter, (x_batch, y_batch) in enumerate(test_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        logits = model(x_batch)

        top1, top5 = accuracy(logits, y_batch, topk=(1,5))
        top1_accuracy += top1[0]
        top5_accuracy += top5[0]

    top1_accuracy /= (counter + 1)
    top5_accuracy /= (counter + 1)
    print(f"Epoch {epoch}\t"
          f"Top1 Train accuracy {top1_train_accuracy.item()}\t"
          f"Top1 Test accuracy: {top1_accuracy.item()}\t"
          f"Top5 test accuracy: {top5_accuracy.item()}")

Epoch 0	Top1 Train accuracy 23.0	Top1 Test accuracy: 70.810546875	Top5 test acc: 95.947265625
Epoch 1	Top1 Train accuracy 92.0	Top1 Test accuracy: 88.92578125	Top5 test acc: 98.974609375
Epoch 2	Top1 Train accuracy 97.0	Top1 Test accuracy: 90.76171875	Top5 test acc: 99.296875
Epoch 3	Top1 Train accuracy 99.0	Top1 Test accuracy: 91.220703125	Top5 test acc: 99.375
Epoch 4	Top1 Train accuracy 100.0	Top1 Test accuracy: 91.30859375	Top5 test acc: 99.462890625
Epoch 5	Top1 Train accuracy 98.0	Top1 Test accuracy: 91.46484375	Top5 test acc: 99.47265625
Epoch 6	Top1 Train accuracy 99.0	Top1 Test accuracy: 91.81640625	Top5 test acc: 99.4921875
Epoch 7	Top1 Train accuracy 100.0	Top1 Test accuracy: 92.197265625	Top5 test acc: 99.47265625
Epoch 8	Top1 Train accuracy 100.0	Top1 Test accuracy: 92.1484375	Top5 test acc: 99.5703125
Epoch 9	Top1 Train accuracy 99.0	Top1 Test accuracy: 92.34375	Top5 test acc: 99.580078125
Epoch 10	Top1 Train accuracy 100.0	Top1 Test accuracy: 92.431640625	Top5 test acc: 